In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

# Load data
playground_gdf = gpd.read_file("playground.geojson")



In [2]:
park_busyness_details_gdf = gpd.read_file("C:/Users/Justh/OneDrive/Documents/MScComputerScience/S3 Research Practicum/DataAnalysis/preprocessing/mapping/park_busyness_details.geojson")

In [24]:
parks_gdf= gpd.read_file("C:/Users/Justh/OneDrive/Documents/MScComputerScience/S3 Research Practicum/DataAnalysis/preprocessing/parks/manhattan_park_details.geojson")

In [23]:
playground_gdf.shape

(382, 48)

In [3]:
pd.set_option('display.max_columns', None)
#display all columns not truncated

In [4]:
park_busyness_details_gdf

,park_id,park_name,taxi_zone_id,busyness,geometry
0,node/11457630561,Kowsky Plaza,13,12778,POINT (-74.01693 40.71186)
1,node/1203709404,Sherman Creek,127,1159,POINT (-73.91834 40.86097)
2,node/3369582883,unknown,142,43270,POINT (-73.98358 40.77133)
3,node/3369582884,unknown,142,43270,POINT (-73.98355 40.77143)
4,node/3369582885,unknown,142,43270,POINT (-73.98353 40.77136)
...,...,...,...,...,...
495,way/973521995,Ahearn Park,232,7243,"POLYGON ((-73.98368 40.71489, -73.98367 40.714..."
496,way/976724943,unknown,113,18757,"POLYGON ((-73.99115 40.72945, -73.99120 40.729..."
497,way/985281628,Seward Park High School - Community Playground,232,7243,"POLYGON ((-73.98959 40.71557, -73.98896 40.715..."
498,way/990936691,Pier 42 Upland Park,232,7243,"POLYGON ((-73.98118 40.71072, -73.98157 40.710..."


In [5]:
park_busyness_details_gdf.explore("busyness", legend=True)

In [6]:
playground_gdf.columns

Index(['id', '@id', 'access', 'addr:street', 'alt_name', 'architect',
       'barrier', 'description', 'dog', 'ele', 'fixme', 'gnis:feature_id',
       'height', 'landuse', 'layer', 'leisure', 'level', 'lit', 'max_age',
       'min_age', 'name', 'name:el', 'name:etymology:wikidata', 'name_alt',
       'note', 'nycdoitt:bin', 'old_name', 'opening_hours', 'operator',
       'operator:wikidata', 'operator:wikipedia', 'playground', '@relations',
       'playground:theme', 'ref', 'source', 'source:geometry', 'source_ref',
       'sport', 'start_date', 'surface', 'toilets:wheelchair', 'type',
       'website', 'website:alternate', 'wheelchair', 'wikidata', 'geometry'],
      dtype='object')

In [7]:
playground_gdf

,id,@id,access,addr:street,alt_name,architect,barrier,description,dog,ele,fixme,gnis:feature_id,height,landuse,layer,leisure,level,lit,max_age,min_age,name,name:el,name:etymology:wikidata,name_alt,note,nycdoitt:bin,old_name,opening_hours,operator,operator:wikidata,operator:wikipedia,playground,@relations,playground:theme,ref,source,source:geometry,source_ref,sport,start_date,surface,toilets:wheelchair,type,website,website:alternate,wheelchair,wikidata,geometry
0,relation/14331634,relation/14331634,None,None,None,None,None,None,None,None,None,None,None,None,None,playground,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,multipolygon,None,None,None,None,"POLYGON ((-73.98244 40.72698, -73.98261 40.727..."
1,relation/16029655,relation/16029655,yes,None,None,None,None,None,None,None,None,None,None,None,None,playground,None,None,None,None,Battery Playscape,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,concrete,None,multipolygon,None,None,None,None,"POLYGON ((-74.01458 40.70199, -74.01460 40.701..."
2,way/108295746,way/108295746,None,None,None,None,fence,None,None,None,None,None,None,None,None,playground,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((-73.94480 40.76892, -73.94485 40.768..."
3,way/108296007,way/108296007,None,None,None,None,None,None,None,None,None,None,None,None,None,playground,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((-73.94846 40.76484, -73.94835 40.764..."
4,way/128582381,way/128582381,yes,None,None,None,fence,None,None,None,None,None,None,None,None,playground,None,None,None,None,Hecksher Playground,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,https://www.centralparknyc.org/locations/hecks...,https://www.nycgovparks.org/parks/central-park...,None,None,"POLYGON ((-73.97786 40.76832, -73.97779 40.768..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,node/10823061568,node/10823061568,None,None,None,None,None,None,None,None,None,None,None,None,None,playground,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.94083 40.85059)
378,node/10846058078,node/10846058078,private,None,None,None,None,None,None,None,None,None,None,None,None,playground,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.93663 40.85370)
379,node/11554790050,node/11554790050,None,None,None,None,None,None,None,None,None,None,None,None,None,playground,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.96656 40.80312)
380,node/11554790051,node/11554790051,None,None,None,None,None,None,None,None,None,None,None,None,None,playground,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.96580 40.80284)


In [8]:
playground_gdf['name']=playground_gdf['name'].fillna('unknown')

In [9]:
playground_gdf.isnull().sum()

id                           0
@id                          0
access                     234
addr:street                381
alt_name                   377
architect                  381
barrier                    379
description                381
dog                        380
ele                        371
fixme                      380
gnis:feature_id            372
height                     381
landuse                    377
layer                      378
leisure                      0
level                      381
lit                        376
max_age                    377
min_age                    378
name                         0
name:el                    381
name:etymology:wikidata    380
name_alt                   381
note                       381
nycdoitt:bin               381
old_name                   380
opening_hours              374
operator                   368
operator:wikidata          381
operator:wikipedia         381
playground                 375
@relatio

In [10]:
playground_gdf['playground'].isnull().sum()

375

In [11]:
playground_details=playground_gdf[['id','leisure','name','geometry']]

In [12]:
playground_details.explore("id", legend=True)

In [13]:
playground_gdf.shape

(382, 48)

In [14]:
park_busyness_details_gdf.shape

(500, 5)

In [15]:
park_busyness_details_gdf.columns

Index(['park_id', 'park_name', 'taxi_zone_id', 'busyness', 'geometry'], dtype='object')

In [16]:
print(playground_gdf.crs)
print(park_busyness_details_gdf.crs)

GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]


In [19]:
# Perform a spatial join to find parks within each taxi zone
playground_in_parks = gpd.sjoin(playground_details, park_busyness_details_gdf, how='inner', predicate='within')

# Rename columns for clarity
playground_in_parks = playground_in_parks.rename(columns={'id':'playground_id','name':'playground_name'})

# Display the resulting DataFrame
print(playground_in_parks.head())
#[['park_name', 'taxi_zone_id']])

       playground_id     leisure      playground_name  \
0  relation/14331634  playground              unknown   
1  relation/16029655  playground    Battery Playscape   
1  relation/16029655  playground    Battery Playscape   
1  relation/16029655  playground    Battery Playscape   
4      way/128582381  playground  Hecksher Playground   

                                            geometry  index_right  \
0  POLYGON ((-73.98244 40.72698, -73.98261 40.727...          306   
1  POLYGON ((-74.01458 40.70199, -74.01460 40.701...          414   
1  POLYGON ((-74.01458 40.70199, -74.01460 40.701...          415   
1  POLYGON ((-74.01458 40.70199, -74.01460 40.701...          416   
4  POLYGON ((-73.97786 40.76832, -73.97779 40.768...          381   

         park_id             park_name taxi_zone_id  busyness  
0   way/25162896  Tompkins Square Park           79     34054  
1    way/5029111          Battery Park           12      1214  
1    way/5029111          Battery Park           1

In [25]:
# Perform a spatial join to find parks within each taxi zone
playground_in_parks2 = gpd.sjoin(playground_details, parks_gdf, how='inner', predicate='within')

# Rename columns for clarity
playground_in_parks2 = playground_in_parks2.rename(columns={'id':'playground_id','name':'playground_name'})

# Display the resulting DataFrame
print(playground_in_parks2.head())
#[['park_name', 'taxi_zone_id']])

             id_left     leisure            name_left  \
0  relation/14331634  playground              unknown   
1  relation/16029655  playground    Battery Playscape   
4      way/128582381  playground  Hecksher Playground   
5      way/144251101  playground              unknown   
7      way/146396824  playground              unknown   

                                            geometry  index_right  \
0  POLYGON ((-73.98244 40.72698, -73.98261 40.727...           54   
1  POLYGON ((-74.01458 40.70199, -74.01460 40.701...           36   
4  POLYGON ((-73.97786 40.76832, -73.97779 40.768...          279   
5  POLYGON ((-73.97278 40.72450, -73.97291 40.724...           94   
7  POLYGON ((-73.97450 40.73061, -73.97457 40.730...          293   

        id_right                       name_right  \
0   way/25162896             Tompkins Square Park   
1    way/5029111                     Battery Park   
4  way/427818536                     Central Park   
5  way/108235842  John V. Lind

In [26]:
playground_in_parks2.shape

(179, 9)

In [28]:
playground_in_parks2.columns

Index(['id_left', 'leisure', 'name_left', 'geometry', 'index_right',
       'id_right', 'name_right', 'website', 'area'],
      dtype='object')

In [29]:
playground_in_parks2.explore("leisure", legend=True)

In [18]:
playground_in_parks.columns

Index(['id', 'leisure', 'name', 'geometry', 'index_right', 'park_id',
       'park_name', 'taxi_zone_id', 'busyness'],
      dtype='object')

In [20]:
playground_in_parks.explore("busyness", legend=True)

In [22]:
playground_in_parks['playground_id'].value_counts().sum()

237